In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
! pip install -q transformers[sentencepiece] fastai ohmeow-blurr nbdev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install -q onnx
!pip install -q onnxruntime
!pip install -q onnxruntime-gpu
!pip install -q onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.8 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import*
from blurr.text.data.all import*
from blurr.text.modeling.all import*

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [6]:
from tqdm.notebook import tqdm
import numpy as np

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd /content/drive/MyDrive/Capstone3

/content/drive/MyDrive/Capstone3


In [9]:
df = pd.read_csv("imdb_movies.csv")
df.head()

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']","After more than thirty years of service as one of the Navy's top aviators, Pete Mitchell is where he belongs, pushing the envelope as a courageous test pilot and dodging the advancement in rank that would ground","['fighter jet', 'sequel', 'u.s. navy', 'fighter aircraft', 'pilot', 'naval aviation', 'jet fighter', 'military pilot', 'aviation', 'aircraft']",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise', 'Jennifer Connelly', 'Miles Teller']",Jim Cash,-2022,/title/tt1745960/
1,Jurassic World Dominion,2 hours 27 minutes,6,56K,"['Action', 'Adventure', 'Sci-Fi']","Four years after the destruction of Isla Nublar, dinosaurs now live--and hunt--alongside humans all over the world. This fragile balance will reshape the future and determine, once and for all, whether human beings are to remain the apex predators on a planet they now share with ... Read all","['dinosaur', 'jurassic park', 'tyrannosaurus rex', 'velociraptor', 'sequel', 'alan grant character', 'ellie sattler character', 'ian malcolm character', 'dinosaur feature', 'animal trainer']",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris Pratt', 'Bryce Dallas Howard', 'Laura Dern']",Emily Carmichael,-2022,/title/tt8041270/
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']","As students at the United States Navy's elite fighter weapons school compete to be best in the class, one daring young pilot learns a few things from a civilian instructor that are not taught in the classroom.","['pilot', 'male camaraderie', 'u.s. navy', 'grumman f 14 tomcat', 'flying', 'jet fighter', 'fighter pilot', 'jet', 'rivalry', 'flight school']",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', 'Tim Robbins', 'Kelly McGillis']",Jim Cash,-1986,/title/tt0092099/
3,Lightyear,"$71,101,257",5.2,32K,"['Animation', 'Action', 'Adventure']","While spending years attempting to return home, marooned Space Ranger Buzz Lightyear encounters an army of ruthless robots commanded by Zurg who are attempting to steal his fuel source.","['galaxy', 'spaceship', 'robot', 'rocket', 'space adventure', 'same sex kiss', 'brainwashing', 'anti conservative', 'spin off', 'pixar']",Angus MacLane,"['Jason Headley', 'Matthew Aldrich', 'Chris Evans', 'Keke Palmer', 'Peter Sohn']",Angus MacLane,-2022,/title/tt10298810/
4,Spiderhead,not-released,5.4,23K,"['Action', 'Crime', 'Drama']","In the near future, convicts are offered the chance to volunteer as medical subjects to shorten their sentence. One such subject for a new drug capable of generating feelings of love begins questioning the reality of his emotions.","['discover', 'medical', 'test', 'reality', 'fictional drug', 'violence', 'room', 'nature', 'emotion', 'volunteer']",Joseph Kosinski,"['Rhett Reese', 'Paul Wernick', 'Chris Hemsworth', 'Miles Teller', 'Jurnee Smollett']",George Saunders,-2022,/title/tt9783600/


In [10]:
df = df.dropna().reset_index(drop=True)
df.shape

(23412, 12)

In [11]:
genres_list = df.Generes.to_list()
genre_count = {}

for genres in genres_list:
  genre_list = eval(genres)
  for genre in genre_list:
    if genre in genre_count.keys():
      genre_count[genre] +=1
    else:
      genre_count[genre] = 1

print(f"Number of Genres: {len(genre_count)}")
print(genre_count)

threshold = int(len(df) * 0.01)
rare_genres = [key for key, value in genre_count.items() if value < threshold]
len(rare_genres)

Number of Genres: 24
{'Action': 7359, 'Drama': 12155, 'Adventure': 4241, 'Sci-Fi': 1979, 'Animation': 779, 'Crime': 5881, 'Comedy': 6701, 'Thriller': 4769, 'Fantasy': 1425, 'Horror': 3144, 'History': 671, 'Mystery': 2038, 'Biography': 974, 'War': 508, 'Western': 593, 'Sport': 345, 'Family': 1074, 'Romance': 3119, 'Music': 446, 'Musical': 316, 'Film-Noir': 309, 'Game-Show': 1, 'Adult': 1, 'Reality-TV': 1}


3

In [12]:
genres_list = df.Generes.to_list()
revised_genre_list = []
indices_to_drop = []

for idx,genres in enumerate(genres_list):
  genre_list = eval(genres)
  revised_genres = []

  for genre in genre_list:
    if genre not in rare_genres:
      revised_genres.append(genre)

  if len(revised_genres) == 0:
    indices_to_drop.append(idx)
  else:
    revised_genre_list.append(revised_genres)



df = df.drop(indices_to_drop).reset_index(drop=True)
df['revised_genres'] = revised_genre_list
df.shape

(23412, 13)

In [13]:
revised_genres_list = df.revised_genres.to_list()
revised_genre_count = {}

for genres in revised_genre_list:
  genre_list = genres
  for genre in genre_list:
    if genre in revised_genre_count.keys():
      revised_genre_count[genre] +=1
    else:
      revised_genre_count[genre] = 1

print(f"Number of Genres: {len(revised_genre_count)}")

Number of Genres: 21


In [14]:
encode_genre_types = {key: idx for idx, (key, value) in enumerate(revised_genre_count.items())}
with open("genre_types_encoded.json", "w") as fp:
  json.dump(encode_genre_types, fp)

categorical_genre_list = []
revised_genres_list = df.revised_genres.to_list()

for revised_genres in revised_genres_list:
  categorical_list = [0] * len(encode_genre_types)
  for genre in revised_genres:
    genre_type_index = encode_genre_types[genre]
    categorical_list[genre_type_index] = 1
  categorical_genre_list.append(categorical_list)

#categorical_genre_list[3][:23]

df['genre_categorical_list'] = categorical_genre_list
df.shape

(23412, 14)

In [15]:
labels = list(encode_genre_types.keys())
len(labels), labels[:5]

(21, ['Action', 'Drama', 'Adventure', 'Sci-Fi', 'Animation'])

# Data Split

In [16]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(21071, 2341)

In [17]:
valid_df = df.loc[valid_ids]
valid_df.head()

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path,revised_genres,genre_categorical_list
22434,Mirror Images,not-released,4.2,428,"['Drama', 'Thriller']","When her identical twin sister goes out of town, a bored and sexually frustrated woman adopts her sibling's promiscuous identity. But soon finds herself involved in a murder case.","['sister sister relationship', 'erotica', 'sexploitation film', 'lesbian character', 'erotic thriller', 'twin', 'nudity', 'assumed identity', 'husband wife relationship', 'lesbianism']",Gregory Dark,"['Delia Sheppard', 'Jeff Conaway', 'Richard Arbolino', 'Gregory Dark', 'Georges des Esseintes']",Georges des Esseintes,-1992,/title/tt0102452/,"[Drama, Thriller]","[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6405,Ek Ajnabee,2 hours 22 minutes,5.3,1.7K,"['Action', 'Crime', 'Drama']","Suryaveer, an ex-army officer, is assigned the task of protecting a young girl, Anamika. When Anamika gets kidnapped, he decides to use all his skills to rescue her.","['betrayal by a friend', 'betrayal', 'abduction', 'telephone conversation', 'little girl', 'kidnapping', 'ransom', 'alcoholic', 'bodyguard', 'murder of a child']",Apoorva Lakhia,"['Manoj Tyagi', 'Amitabh Bachchan', 'Arjun Rampal', 'Perizaad Zorabian', 'Apoorva Lakhia']",Apoorva Lakhia,-2005,/title/tt0457875/,"[Action, Crime, Drama]","[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
21038,Deadly Girls Night Out,not-released,5.5,95,['Thriller'],"Deadly Girls Night Out is the story of three close friends planning a party out for the night. However, the night ends up in a tragic death that unravel deep secrets.","[""girls' night out"", 'grammatical error in title']",Ben Meyerson,"['Gina Vitori', 'Margo Parker', 'James Hyde', 'Ben Meyerson', 'Justin D James']",Justin D James,-2021,/title/tt15739916/,[Thriller],"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
16522,The Three Investigators and the Secret of Terror Castle,1 hour 37 minutes,5.6,1.1K,"['Adventure', 'Family', 'Mystery']","When the case he's working on leads straight to Terror Castle, boy detective Jupiter Jones enlists the help of his two friends. Together they investigate the castle's many secrets, but end up finding more than they bargained for.","['covered in syrup', 'steampunk', 'haunted house', 'disguise', 'young detectives', 'based on novel', 'party', 'father son relationship', 'gift']",Florian Baxmeyer,"['Aaron Mendelsohn', 'Robert Arthur', 'Chancellor Miller', 'Nick Price', 'Cameron Monaghan']",Philip LaZebnik,-2009,/title/tt1156519/,"[Adventure, Family, Mystery]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
8366,Beethoven,1 hour 27 minutes,5.7,71K,"['Comedy', 'Drama', 'Family']","A slobbering St. Bernard becomes the center of attention for a loving family, but must contend with a dog-napping veterinarian and his henchmen.","['dog', 'animal', 'husband wife relationship', 'mother', 'father', 'pet dog', 'family relationships', 'dysfunctional family', 'animal actor', 'rescue']",Brian Levant,"['Amy Holden Jones', 'Charles Grodin', 'Bonnie Hunt', 'Dean Jones', 'Brian Levant']",John Hughes,-1992,/title/tt0103786/,"[Comedy, Drama, Family]","[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"


# Fastai & Blurr Inference

In [18]:
model_path = "models/movie-classifier-stage-0.pkl"
learner_inf = load_learner(model_path)

In [19]:
learner_inf.blurr_predict("random placeholder")

[{'labels': ['Comedy'],
  'scores': [0.6310023069381714],
  'class_indices': [0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'class_labels': ['Action', 'Drama', 'Adventure', 'Sci-Fi', 'Animation', 'Crime', 'Comedy', 'Thriller', 'Fantasy', 'Horror', 'History', 'Mystery', 'Biography', 'War', 'Western', 'Sport', 'Family', 'Romance', 'Music', 'Musical', 'Film-Noir'],
  'probs': [0.2765035629272461,
   0.1473991870880127,
   0.19879695773124695,
   0.3692411482334137,
   0.11463603377342224,
   0.16896618902683258,
   0.6310023069381714,
   0.10352838039398193,
   0.12281092256307602,
   0.10952378064393997,
   0.008469299413263798,
   0.055518705397844315,
   0.006198083981871605,
   0.005485530011355877,
   0.009435656480491161,
   0.011670735664665699,
   0.09010611474514008,
   0.10739731043577194,
   0.011907647363841534,
   0.02019936591386795,
   0.0068011474795639515]}]

In [20]:
learner_inf.blurr_predict("random placeholder")[0]['labels']

['Comedy']

# Evaluations

In [21]:
from sklearn import metrics

def metric_measures(test_df, preds):
  targets = [np.asarray(target) for target in test_df['genre_categorical_list'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]

  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [22]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['Overview']
  labels = learner_inf.blurr_predict(desc)[0]['class_indices']
  preds.append(labels)

preds[0][:20]

  0%|          | 0/2341 [00:00<?, ?it/s]

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [23]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.5766181635725037
F1 Score (Macro) = 0.41747256327432536


# Convert to ONNX

In [24]:
model_path = "models/movie-classifier-stage-0.pkl"
learner_inf = load_learner(model_path)

In [25]:
learner_inf.model.hf_model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [27]:
classifier = learner_inf.model.hf_model.eval()
torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    'models/movie-classifier.onnx',
    verbose=True,
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [32]:
from onnxruntime.quantization import quantize_dynamic, QuantType


onnx_model_path = 'models/movie-classifier.onnx'
#quantize_onnx_model_path = 'models/movie-classifier-quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    #quantized_onnx_model_path,
    weight_type=QuantType.QUInt8
)

TypeError: quantize_dynamic() missing 1 required positional argument: 'model_output'

# Normal ONNX

In [33]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession('models/movie-classifier.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [34]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['Overview']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/2341 [00:00<?, ?it/s]

In [35]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.5751633986928105
F1 Score (Macro) = 0.41326616719319675
